In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_error

import sys, os
sys.path.append(os.path.abspath(".."))
from utils.updateStats import getStats, updateStats, createStats
pd.set_option('display.max_columns', None)

In [2]:
allData = pd.read_csv("../data/0cleanDatasetWithQualifiersWith2025.csv")
allData

/var/folders/72/931mm5912h7_y6ts82j2wjhh0000gn/T/ipykernel_52750/2420848872.py:1: DtypeWarning: Columns (6,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  allData = pd.read_csv("../data/0cleanDatasetWithQualifiersWith2025.csv")


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p2_rank,RESULT
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,2.0,1
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,6-3 3-6 7-6(6),3,R32,119.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,304.0,75.0,1
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,6-0 6-4,3,R32,71.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,69.0,82.0,0
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,7-6(2) 6-1,3,R32,85.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,84.0,50.0,0
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,7-5 6-3,3,R32,90.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,28.0,88.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197258,2025-06-23_Milan CH_2025-3463-377,Milan CH,Clay,32.0,C,20250623,2025-3463-377,123809,NaN,NaN,Ivan Gakhov,L,191.0,RUS,28.6,122352,NaN,NaN,Christoph Negritu,R,193.0,GER,31.3,6-3 6-4,3,R16,98.0,4.0,6.0,54.0,32.0,24.0,6.0,9.0,2.0,5.0,6.0,2.0,85.0,54.0,40.0,12.0,10.0,9.0,10.0,270.0,290.0,0
197259,2025-06-23_Milan CH_2025-3463-378,Milan CH,Clay,32.0,C,20250623,2025-3463-378,209976,NaN,NaN,Dino Prizmic,R,188.0,CRO,19.9,202235,NaN,NaN,Jacopo Berrettini,R,193.0,ITA,26.6,6-1 6-2,3,R16,84.0,2.0,5.0,46.0,22.0,16.0,13.0,8.0,3.0,5.0,0.0,3.0,41.0,20.0,7.0,8.0,7.0,1.0,7.0,173.0,330.0,1
197260,2025-06-23_Milan CH_2025-3463-380,Milan CH,Clay,32.0,C,20250623,2025-3463-380,106065,NaN,NaN,Marco Cecchinato,R,185.0,ITA,32.7,208069,NaN,NaN,Max Houkes,L,188.0,NED,25.0,6-4 6-2,3,QF,92.0,1.0,3.0,45.0,30.0,26.0,11.0,9.0,2.0,2.0,3.0,6.0,63.0,35.0,22.0,13.0,9.0,2.0,5.0,437.0,256.0,1
197261,2025-06-23_Milan CH_2025-3463-382,Milan CH,Clay,32.0,C,20250623,2025-3463-382,209976,NaN,NaN,Dino Prizmic,R,188.0,CRO,19.9,122352,NaN,NaN,Christoph Negritu,R,193.0,GER,31.3,6-4 6-3,3,QF,94.0,1.0,6.0,64.0,28.0,20.0,17.0,10.0,3.0,6.0,0.0,0.0,45.0,29.0,15.0,6.0,9.0,1.0,6.0,173.0,290.0,1


In [3]:
def run_ELO_evaluation(evaluation_data, prev_stats, k_factor, base_k_factor, max_k_factor, div_number, bonus_after_layoff, warmup_n: int = 10000):
    def calculate_accuracy_score(preds, labels):
        return accuracy_score(labels, preds) if len(labels) > 0 else float('nan')

    majors = {"A", "G", "M", "F", "D", "O"}

    elo_preds, elo_preds_surface, labels = [], [], []
    elo_preds_2025, elo_preds_surface_2025, labels_2025 = [], [], []

    for i, (_, row) in enumerate(tqdm(evaluation_data.iterrows(), total=len(evaluation_data))):
        # Initial ELO by level
        init_elo = 1500

        # Current ratings
        elo1  = prev_stats["elo_players"].get(row.p1_id, init_elo)
        elo2  = prev_stats["elo_players"].get(row.p2_id, init_elo)
        elo1s = prev_stats["elo_surface_players"][row.surface].get(row.p1_id, init_elo)
        elo2s = prev_stats["elo_surface_players"][row.surface].get(row.p2_id, init_elo)

        # Predict on majors after warm-up
        if row.tourney_level in majors and i > warmup_n:
            is_2025 = str(row.tourney_date).startswith("2025")
            p  = 1 if elo1  >= elo2  else 0
            ps = 1 if elo1s >= elo2s else 0

            if is_2025:
                elo_preds_2025.append(p)
                elo_preds_surface_2025.append(ps)
                labels_2025.append(row.RESULT)
            else:
                elo_preds.append(p)
                elo_preds_surface.append(ps)
                labels.append(row.RESULT)

        # Update after predicting
        prev_stats = updateStats(row, prev_stats, k_factor, base_k_factor, max_k_factor, div_number, bonus_after_layoff)

    global_elo_acc           = calculate_accuracy_score(elo_preds, labels)
    global_elo_surface_acc   = calculate_accuracy_score(elo_preds_surface, labels)
    elo_acc_2025             = calculate_accuracy_score(elo_preds_2025, labels_2025)
    elo_surface_acc_2025     = calculate_accuracy_score(elo_preds_surface_2025, labels_2025)

    print(
        f"\nEVALUATION RESULTS:\n"
        f"Global ELO Accuracy: {global_elo_acc} ({len(labels)} matches)\n"
        f"2025 ELO Accuracy: {elo_acc_2025} ({len(labels_2025)} matches)\n"
        f"Global ELO Surface Accuracy: {global_elo_surface_acc} ({len(labels)} matches)\n"
        f"2025 ELO Surface Accuracy: {elo_surface_acc_2025} ({len(labels_2025)} matches)\n"
    )
    return global_elo_acc, elo_acc_2025, global_elo_surface_acc, elo_surface_acc_2025


In [4]:
########## TEST 1 ##########
# Vanilla ELO K factor = 24
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                   prev_stats=prev_stats_eval, 
                   k_factor=24, 
                   base_k_factor=0, 
                   max_k_factor=0, 
                   div_number=0, 
                   bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:13<00:00, 14922.43it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6564919696512228 (102673 matches)
2025 ELO Accuracy: 0.6419813634134379 (2039 matches)
Global ELO Surface Accuracy: 0.647560702424201 (102673 matches)
2025 ELO Surface Accuracy: 0.6390387444825895 (2039 matches)



In [5]:
########## TEST 2 ##########
# Vanilla ELO K factor = 32
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                   prev_stats=prev_stats_eval, 
                   k_factor=32,
                   base_k_factor=0,
                   max_k_factor=0,
                   div_number=0,
                   bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:13<00:00, 14677.87it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6567549404419857 (102673 matches)
2025 ELO Accuracy: 0.6459048553212359 (2039 matches)
Global ELO Surface Accuracy: 0.6494209772773758 (102673 matches)
2025 ELO Surface Accuracy: 0.6463952918097107 (2039 matches)



In [6]:
########## TEST 3 ##########
# Vanilla ELO K factor = 50
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                   prev_stats=prev_stats_eval, 
                   k_factor=50,
                   base_k_factor=0,
                   max_k_factor=0,
                   div_number=0,
                   bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:13<00:00, 14603.39it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.655157636379574 (102673 matches)
2025 ELO Accuracy: 0.6508092202059833 (2039 matches)
Global ELO Surface Accuracy: 0.6513396900840532 (102673 matches)
2025 ELO Surface Accuracy: 0.6522805296714076 (2039 matches)



In [7]:
########## TEST 4 ##########
# Vanilla ELO K factor = 100
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData,
                   prev_stats=prev_stats_eval,
                   k_factor=100,
                   base_k_factor=0,
                   max_k_factor=0,
                   div_number=0,
                   bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:13<00:00, 14491.85it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6468010090286638 (102673 matches)
2025 ELO Accuracy: 0.6414909269249632 (2039 matches)
Global ELO Surface Accuracy: 0.6474925248117811 (102673 matches)
2025 ELO Surface Accuracy: 0.6434526728788622 (2039 matches)



In [8]:
########## TEST 5 ##########
# ELO with base_k_factor=20, max_k_factor=60
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                   prev_stats=prev_stats_eval, 
                   k_factor=None,
                   base_k_factor=20,
                   max_k_factor=60,
                   div_number=600,
                   bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:13<00:00, 14168.28it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6576899476980316 (102673 matches)
2025 ELO Accuracy: 0.6414909269249632 (2039 matches)
Global ELO Surface Accuracy: 0.6521188627974248 (102673 matches)
2025 ELO Surface Accuracy: 0.6478666012751348 (2039 matches)



In [9]:
########## TEST 6 ##########
# ELO with base_k_factor=40, max_k_factor=80
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                   prev_stats=prev_stats_eval, 
                   k_factor=None,
                   base_k_factor=40,
                   max_k_factor=80,
                   div_number=600,
                   bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:14<00:00, 14000.81it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6561900402247913 (102673 matches)
2025 ELO Accuracy: 0.6493379107405591 (2039 matches)
Global ELO Surface Accuracy: 0.6525863664254478 (102673 matches)
2025 ELO Surface Accuracy: 0.6562040215792055 (2039 matches)



In [10]:
########## TEST 7 ##########
# ELO with base_k_factor=40, max_k_factor=60, div_number=600
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData,
                         prev_stats=prev_stats_eval,
                         k_factor=None,
                         base_k_factor=40,
                         max_k_factor=60,
                         div_number=600,
                         bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:13<00:00, 14129.80it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6563458747674656 (102673 matches)
2025 ELO Accuracy: 0.6493379107405591 (2039 matches)
Global ELO Surface Accuracy: 0.6520019868904191 (102673 matches)
2025 ELO Surface Accuracy: 0.6576753310446297 (2039 matches)



In [11]:
########## TEST 8 ##########
# ELO with base_k_factor=40, max_k_factor=80, div_number=800
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                         prev_stats=prev_stats_eval, 
                         k_factor=None, 
                         base_k_factor=40, 
                         max_k_factor=80, 
                         div_number=800,
                         bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:14<00:00, 13992.28it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6562095195426256 (102673 matches)
2025 ELO Accuracy: 0.6493379107405591 (2039 matches)
Global ELO Surface Accuracy: 0.6524889698362764 (102673 matches)
2025 ELO Surface Accuracy: 0.657184894556155 (2039 matches)



In [12]:
########## TEST 9 ##########
# ELO with base_k_factor=40, max_k_factor=80, div_number=800 + K factor varied when tennis player come back
############################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                         prev_stats=prev_stats_eval, 
                         k_factor=None, 
                         base_k_factor=40, 
                         max_k_factor=80, 
                         div_number=800,
                         bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:14<00:00, 13988.34it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6562095195426256 (102673 matches)
2025 ELO Accuracy: 0.6493379107405591 (2039 matches)
Global ELO Surface Accuracy: 0.6524889698362764 (102673 matches)
2025 ELO Surface Accuracy: 0.657184894556155 (2039 matches)



## Custom GridSearch

In [ ]:
# from scipy.stats import loguniform, randint
# import random

# param_dist = {
#     "base_k_factor": randint(0, 50),
#     "max_k_factor": randint(50, 100),
#     "div_number": randint(500, 1000)
# }

# N_ITER = 300
# best_score = -np.inf
# best_params = None

# elo_df = pd.DataFrame(columns=["ELO_GLOBAL_SCORE", "ELO_2025_SCORE", "ELO_SURFACE_GLOBAL_SCORE", "ELO_SURFACE_2025_SCORE", "base_k_factor", "max_k_factor", "div_number"])

# for i in tqdm(range(N_ITER)):
#     # Randomly sample parameters
#     params = {k: v.rvs() for k, v in param_dist.items()}

#     prev_stats_eval = createStats()
#     elo_global_score, elo_2025_score, elo_surface_global_score, elo_surface_2025_score = run_ELO_evaluation(evaluation_data=allData, prev_stats=prev_stats_eval, k_factor=None, **params)
    
#     new_row = {"ELO_GLOBAL_SCORE": elo_global_score, "ELO_2025_SCORE": elo_2025_score, "ELO_SURFACE_GLOBAL_SCORE": elo_surface_global_score, "ELO_SURFACE_2025_SCORE": elo_surface_2025_score,  **params}
    
#     elo_df.loc[len(elo_df)] = new_row

  0%|          | 1/300 [00:14<1:10:14, 14.10s/it]


EVALUATION RESULTS:
Global ELO Accuracy: 0.6559443460501332 (62673 matches)
2025 ELO Accuracy: 0.6365865620402158 (2039 matches)
Global ELO Surface Accuracy: 0.6470250347039395 (62673 matches)
2025 ELO surface Accuracy: 0.6405100539480137 (2039 matches)



  1%|          | 2/300 [00:28<1:11:01, 14.30s/it]


EVALUATION RESULTS:
Global ELO Accuracy: 0.661225727187146 (62673 matches)
2025 ELO Accuracy: 0.6493379107405591 (2039 matches)
Global ELO Surface Accuracy: 0.6561517718953935 (62673 matches)
2025 ELO surface Accuracy: 0.6522805296714076 (2039 matches)



  1%|          | 3/300 [00:42<1:10:35, 14.26s/it]

...


EVALUATION RESULTS:
Global ELO Accuracy: 0.6583696328562539 (62673 matches)
2025 ELO Accuracy: 0.6410004904364884 (2039 matches)
Global ELO Surface Accuracy: 0.6514288449571586 (62673 matches)
2025 ELO surface Accuracy: 0.6429622363903874 (2039 matches)



100%|█████████▉| 299/300 [28:17:45<08:20, 500.71s/it]


EVALUATION RESULTS:
Global ELO Accuracy: 0.6587047053755205 (62673 matches)
2025 ELO Accuracy: 0.6434526728788622 (2039 matches)
Global ELO Surface Accuracy: 0.649689659023822 (62673 matches)
2025 ELO surface Accuracy: 0.6434526728788622 (2039 matches)



100%|██████████| 300/300 [28:18:35<00:00, 339.72s/it]


EVALUATION RESULTS:
Global ELO Accuracy: 0.6619596955626825 (62673 matches)
2025 ELO Accuracy: 0.6488474742520843 (2039 matches)
Global ELO Surface Accuracy: 0.6560241252213872 (62673 matches)
2025 ELO surface Accuracy: 0.6542422756253066 (2039 matches)



Unfortunately, I didn't save these results to a CSV. If you want to, you can uncomment the above cell and run it. Otherwise, just scroll to the end of this notebook to see the optimal ELO hyperparameters :)

In [ ]:
# elo_df.sort_values("ELO_GLOBAL_SCORE", ascending=False).head(50)

In [ ]:
# elo_df.sort_values("ELO_GLOBAL_SCORE", ascending=False).head(50)

,ELO_GLOBAL_SCORE,ELO_2025_SCORE,ELO_SURFACE_GLOBAL_SCORE,ELO_SURFACE_2025_SCORE,base_k_factor,max_k_factor,div_number
0,0.660954,0.643943,0.650966,0.642962,16,79,610


In [ ]:
# elo_df.sort_values("Score2025", ascending=False).head(30)

,AllDataScore,Score2025,base_k_factor,max_k_factor
29,0.661162,0.651790,43,62
213,0.661353,0.651300,44,89
250,0.660492,0.651300,46,56
241,0.661401,0.650809,43,72
266,0.661162,0.650809,44,92
233,0.661417,0.650809,43,73
199,0.660444,0.650809,46,57
86,0.660492,0.650809,46,58
272,0.661098,0.650809,44,77
48,0.660875,0.650809,47,71


In [ ]:
# elo_df.sort_values("AllDataScore", ascending=False).head(30)

,AllDataScore,Score2025,base_k_factor,max_k_factor
138,0.662359,0.646395,38,80
161,0.662359,0.646395,33,82
295,0.662327,0.647867,32,82
214,0.662311,0.646886,38,88
60,0.662311,0.648357,32,97
286,0.662295,0.648847,36,95
181,0.662279,0.643453,24,97
95,0.662231,0.647867,38,79
186,0.662215,0.644924,39,97
5,0.662215,0.646395,38,77


In [13]:
########## Test with best params ##########
# ELO with base_k_factor=38, max_k_factor=80
###########################################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                         prev_stats=prev_stats_eval, 
                         k_factor=None, 
                         base_k_factor=43, 
                         max_k_factor=62, 
                         div_number=800,
                         bonus_after_layoff=False)
del prev_stats_eval

100%|██████████| 197263/197263 [00:13<00:00, 14312.56it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.6556056606897627 (102673 matches)
2025 ELO Accuracy: 0.6512996566944581 (2039 matches)
Global ELO Surface Accuracy: 0.6522746973400991 (102673 matches)
2025 ELO Surface Accuracy: 0.6566944580676802 (2039 matches)



In [14]:
########## Test with best params ##########
# ELO with base_k_factor=38, max_k_factor=80 + bonus_after_layoff
###########################################

prev_stats_eval = createStats()
run_ELO_evaluation(evaluation_data=allData, 
                         prev_stats=prev_stats_eval, 
                         k_factor=None, 
                         base_k_factor=43, 
                         max_k_factor=62, 
                         div_number=800,
                         bonus_after_layoff=True)
del prev_stats_eval

100%|██████████| 197263/197263 [01:06<00:00, 2947.97it/s]



EVALUATION RESULTS:
Global ELO Accuracy: 0.655498524441674 (102673 matches)
2025 ELO Accuracy: 0.6522805296714076 (2039 matches)
Global ELO Surface Accuracy: 0.6517682350764076 (102673 matches)
2025 ELO Surface Accuracy: 0.6552231486022561 (2039 matches)



Cool from now on we'll use the following hyperparameters when updating the stats:

```
updateStats(row, prev_stats, k_factor=None, base_k_factor=43, max_k_factor=62, div_number=800, bonus_after_layoff=True)
```